In [81]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
% matplotlib inline

import sys, os, copy
print("Using environment in "+sys.prefix)
print("Python version "+sys.version)

Using environment in /Users/emunsing/Documents/Coding/github/cdips_hpi_forecast/env
Python version 3.5.2 (default, Oct 31 2016, 16:50:28) 
[GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)]


In [2]:
fname = 'Metro_Zhvi_AllHomes.csv'

#  Map Zillow RegionID to metro area string
metroRegionID = {'Dallas-Fort Worth, TX':394514,
                     'Atlanta, GA':394347,
                     'Phoenix, AZ':394976,
                     'Las Vegas, NV':394775 }

metroString = {'Dallas':u'Dallas-Fort Worth, TX',
               'Atlanta':u'Atlanta, GA',
               'Phoenix':u'Phoenix, AZ',
               'Vegas':u'Las Vegas, NV'}

df = pd.read_csv('Data/ZHVI/'+fname,index_col=[1])
df = df.drop(['RegionID','SizeRank'],axis=1)
df.columns = pd.DatetimeIndex(df.columns)


# Nearest-neighbor approach

Look for the towns that provide good leading indicators for each city of interest. Would like to say "Hey, the albequerque market just went through a drop last month; we'll probably be doing the same this month"

- Drop all the datasets which don't have data for 2004 onwards

For each city
- Save the target series
- Shift all the other data by x months
- Compute the nearest neighbor between the target and the shifted data
- Save the RMSE and city for that lag

Repeat for different shift lengths (different values of x).

In [15]:
# Drop all dates that don't have the minimum amount of data
dataStartsBefore = 2004
minMonths = (2017 - dataStartsBefore) * 12
df = df.drop(df.index[df.count(axis=1) < minMonths], axis=0)
df.shape

(481, 254)

In [80]:
cityList = list(metroString.keys())
# For each shift period, we want to store the quality of the fit, and the city which is the best fit
resultRMSE = pd.DataFrame()  # We'll get this RMSE...
resultCity = pd.DataFrame()  # ...when we use this city

for city in cityList:
    # Compute the 6-month percent differences for our target city...
    ts = df.loc[metroString[city],:]
    ts = ts.shift(-6)/ts - 1
    ts = ts * 100

    # And compute the 6-month percent differences for everywhere else.
    compareWith = df.drop(metroString[city],axis=0)
    compareWith = compareWith.shift(-6,axis=1)/compareWith - 1
    compareWith = compareWith * 100
    
    for shiftBy in range(1,7):
        # Shift the reference dataset, compute the RMSE relative to the actual, and save the results
        shifted = compareWith.shift(shiftBy,axis=1)
        diff = shifted - ts
        
        rmse = (np.sqrt(diff.pow(2).sum(axis=1) / diff.count(axis=1)))  # Note: Diff.count() will exclude NaNs
        resultRMSE.loc[city, shiftBy] = rmse.min()
        resultCity.loc[city, shiftBy] = rmse.idxmin()
        
resultRMSE['best'] = resultRMSE.min(axis=1)
resultRMSE['bestLag'] = resultRMSE.idxmin(axis=1)

for thisCity in resultCity.index.values:
    resultRMSE.loc[thisCity,'bestLoc'] = resultCity.loc[thisCity,resultRMSE.loc[thisCity,'bestLag']]
    
resultRMSE


,1,2,3,4,5,6,best,bestLag,bestLoc
Atlanta,2.144748,2.177312,2.146770,2.122897,2.125478,2.121102,2.121102,6,"Cleveland, OH"
Phoenix,3.729301,3.962419,3.979273,3.900940,3.905888,4.000408,3.729301,1,"Tucson, AZ"
Dallas,2.449048,2.451128,2.427014,2.396366,2.352487,2.286828,2.286828,6,"Lincoln, NE"
Vegas,4.607465,4.644182,4.754334,4.923742,5.152675,5.440050,4.607465,1,"Riverside, CA"
